# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Sean Chapman</p>
### <p style="text-align: right;"> &#9989; chapm348</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone https://github.com/chapm348/CMSE202-s23-turnin

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [1]:
#Write your code here
!curl -O https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4636k  100 4636k    0     0   908k      0  0:00:05  0:00:05 --:--:-- 1141k


In [50]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import statsmodels.api as sm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn import __version__ as sk_version
from sklearn.metrics import classification_report
import time

In [4]:

weather_df = pd.read_csv("aussie_weather_data.csv")
weather_df.head()

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [5]:
#Write your code here
weather_df["Location"].unique()

array(['NorfolkIsland', 'SydneyAirport', 'WaggaWagga', 'MelbourneAirport',
       'Mildura', 'Watsonia', 'Brisbane', 'Cairns', 'Townsville',
       'MountGambier', 'Nuriootpa', 'PerthAirport', 'Perth',
       'AliceSprings', 'Darwin'], dtype=object)

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [6]:
#Write your code here

mask = (weather_df["Location"]=="Cairns")
Cairns_df = weather_df[mask]
Cairns_df.head()


,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
18956,2008,12,1,Cairns,25.2,32.3,0.4,4.0,6.4,67.5,...,62.0,66.0,1011.3,1008.1,5.0,7.0,31.2,30.7,0,1
18957,2008,12,2,Cairns,24.2,32.3,8.4,6.6,5.3,67.5,...,75.0,62.0,1011.9,1008.0,7.0,7.0,29.0,31.1,1,0
18958,2008,12,3,Cairns,23.9,32.8,0.6,5.6,11.3,292.5,...,69.0,42.0,1012.2,1009.9,7.0,5.0,29.1,31.6,0,0
18959,2008,12,4,Cairns,22.2,33.6,0.0,9.6,12.3,315.0,...,60.0,31.0,1013.3,1010.9,2.0,1.0,29.6,32.5,0,0
18960,2008,12,5,Cairns,23.2,33.5,0.0,11.6,11.8,315.0,...,64.0,56.0,1013.8,1011.0,4.0,2.0,29.7,32.2,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [67]:
#Write your code here

# features
X = Cairns_df.drop(columns=["RainTomorrow","Location"]) 

# target values
y = Cairns_df["RainTomorrow"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1000)

# making sure test is at 20% of dataset
print(len(X_train))
print(len(X_test))

1955
489


## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

**Logistic regression makes the most sense here because we are dealing with binary variables, since the RainTomorrow variable is either 0 or 1. Also, the dependent variable we are interested in is not continuous, meaning linear reggression probably should not be applied.**

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [68]:
#Write your code here

x_const_test = sm.add_constant(X_test)

logit_model = sm.Logit(y_train, sm.add_constant(X_train))
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.372266
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 1955
Model:                          Logit   Df Residuals:                     1931
Method:                           MLE   Df Model:                           23
Date:                Fri, 21 Apr 2023   Pseudo R-squ.:                  0.3987
Time:                        23:42:40   Log-Likelihood:                -727.78
converged:                       True   LL-Null:                       -1210.3
Covariance Type:            nonrobust   LLR p-value:                3.584e-189
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -48.0252     67.573     -0.711      0.477    -180.467      84.416
Year             -0.

*Record which features are most statistically significant.*
**The statistically significant variables are MinTemp, Sunshine, WindGustSpeed, Humidity3pm, Pressure9am, Pressure3pm, Temp3pm, and RainToday. This is because all have p values < 0.05.**

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [69]:
#Write your code here
model = LogisticRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))

[[303  41]
 [ 51  94]]


/home/chapm348/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

In [70]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87       344
           1       0.70      0.65      0.67       145

    accuracy                           0.81       489
   macro avg       0.78      0.76      0.77       489
weighted avg       0.81      0.81      0.81       489



**Well, for 0 (False), recall is greater since its value is at 0.91 compared to a precision score of 0.86. However, for 1 (True), precision is greater since its value is at 0.70 compared to a recall score of 0.59. What this means, is that all the time sthat the model predicted Rain Tomorrow being True, it was correct 70% of the time. With recall, however, with a score of 0.59, it means that out of all the times RainTomorrow=True actually occured, my model correctly identified it 59% of the time.**

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [49]:
#Write your code for calculating the frequency of rain here.

Cairns_rain_count = 0
Cairns_norain_count = 0

Cairns_rain = (Cairns_df["RainToday"] == 1)

for i in Cairns_rain:
    if (i == True):
        Cairns_rain_count += 1
    if (i == False):
        Cairns_norain_count += 1

Cairns_rain_total = Cairns_rain_count + Cairns_norain_count
Cairns_result = Cairns_rain_count/Cairns_rain_total

print("Cairns rain count (days):", Cairns_rain_count)
print("Cairns day total:", Cairns_rain_total)

print("Frequency of rain in Cairns:", Cairns_result)

Cairns rain count (days): 744
Cairns day total: 2444
Frequency of rain in Cairns: 0.3044189852700491


**Since it rains about 30.4% of the time in Cairns, it can be said that I would be right 69.6% of the time.**

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [71]:
#Write your code here

# Train a SVM classification model

start = time.time()

#make some temporary variables so you can change this easily
tmp_vectors = X_train
tmp_labels = y_train

print("Fitting the classifier to the training set")
# a dictionary of hyperparameters: key is the name of the parameter, value is a list of values to test
param_grid = {'C': [0.01, 0.1, 1.0, 10.0],
              'gamma': [0.01, 0.1, 1.0, 10.0],
              'kernel': ['linear','rbf']}
# make a classifier by searching over a classifier and the parameter grid
clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, n_jobs=-1)

# we have a "good" classifier (according to GridSearchCV), how's it look
clf = clf.fit(tmp_vectors, tmp_labels)
print("Best estimator found by grid search:")
print(clf.best_estimator_)
print("Best parameters found by grid search:")
print(clf.best_params_)

end = time.time()
print("Runtime",end - start)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=0.01, class_weight='balanced', gamma=0.01, kernel='linear')
Best parameters found by grid search:
{'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}
Runtime 5.645813703536987


**The best-fit hyperparameters are when C=0.01, gamma=0.01, and kernel=linear.**

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [72]:
#Write your code here

# Quantitative evaluation of the model quality on the test set

#make some temporary variables so you can change this easily
predict_vectors = X_test
true_labels = y_test

print("Predicting names on the test set")
pred_labels = clf.predict(predict_vectors)

print(classification_report(true_labels, pred_labels))
print(confusion_matrix(true_labels, pred_labels))

Predicting names on the test set
              precision    recall  f1-score   support

           0       0.91      0.76      0.83       344
           1       0.59      0.81      0.68       145

    accuracy                           0.78       489
   macro avg       0.75      0.79      0.75       489
weighted avg       0.81      0.78      0.78       489

[[261  83]
 [ 27 118]]


## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


**The accuracy of my SVM model is 78%, and the accuracy of my regression model is 83%, and my baseline model produced 69.6%. My predictive models certainly do better than my baseline model, with my regression outperforming the SVM model. This is what I would expect, with about a 10% increase in performance. This is also a good measure of my own models, because it means I am on the right track (being above the baseline model).**

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Write down the accuracy, precision, recall, and seed number for each of your iterations.*

**FOR Random state of 20**

**1) Regression. For False Vlaue: Precision = 0.86, Recall = 0.91.** 

**- For Truth Value: Precision = 0.7, Recall = 0.59. Accuracy is 0.83.**

**2) SVM. For False Value: Precision = 0.91, Recall = 0.78.**

**- For Turth Value: Precision = 0.55, Recall = 0.77, accuracy = 0.78.**

**FOR Random state of 45**

**1) Regression. For False Vlaue: Precision = 0.83, Recall = 0.86.** 

**- For Truth Value: Precision = 0.75, Recall = 0.68. Accuracy is 0.81.**

**2) SVM. For False Value: Precision = 0.89, Recall = 0.79.**

**- For Turth Value: Precision = 0.66, Recall = 0.80, accuracy = 0.79.**

**FOR Random state of 90**

**1) Regression. For False Vlaue: Precision = 0.88, Recall = 0.89.** 

**- For Truth Value: Precision = 0.73, Recall = 0.70. Accuracy is 0.84.**

**2) SVM. For False Value: Precision = 0.92, Recall = 0.76.**

**- For Turth Value: Precision = 0.58, Recall = 0.84, accuracy = 0.78.**

**FOR Random state of 1000**

**1) Regression. For False Vlaue: Precision = 0.86, Recall = 0.88.** 

**- For Truth Value: Precision = 0.70, Recall = 0.65. Accuracy is 0.81.**

**2) SVM. For False Value: Precision = 0.91, Recall = 0.76.**

**- For Turth Value: Precision = 0.59, Recall = 0.81, accuracy = 0.78.**

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

**To start, I will note how the accuracy does not fluctuate too heavily. The best accuracy I received was with a random state of 90. It appears that the larger random state I inputed, the worse precision values I got. So, there was actual variation within the precision/recall values, but the accuracy's stayed relatively the same. This does not really chagne my response to part 5.1** 

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

**Since the regression model's best accuracy score was 0.84, it beats out any SVM score I received. The regression model's also received better Precision scores by a good amount, and that can be seen in my listed data above. Finally, the regression model's also received better Recall scores, again by a good amount. Clearly, the regression model outperformed the SVM model here.**

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University